In [322]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import itertools as itr
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTEN
import seaborn as sns

In [ ]:
data=pd.read_csv('data.csv')


PREPROCESSING

In [350]:
cat_columns=['pr_status',  'pam50_+_claudin-low_subtype', 'primary_tumor_laterality',   'inferred_menopausal_state',   'her2_status',   'er_status',   'er_status_measured_by_ihc',   '3-gene_classifier_subtype', 'death_from_cancer']


def one_hot_encode_columns(df, columns,isTrain):
  enc=OneHotEncoder(sparse_output=False)
  encoded_data=enc.fit_transform(df[columns])
  encoded_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names_out(columns))
  df_encoded = pd.concat([df.reset_index(drop=True),encoded_df.reset_index(drop=True)],axis=1)
  return  df_encoded.drop(columns,axis=1)

def data_preprocess(df,isTrain):
  
  
  
  df = df.drop('patient_id', axis=1)
  df=df.drop('cohort',axis=1)
  
  
  numerical_columns = ['neoplasm_histologic_grade', 'mutation_count', 'tumor_size', 'tumor_stage']
  df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())

  categorical_columns = ['er_status_measured_by_ihc', 'primary_tumor_laterality', '3-gene_classifier_subtype', 'death_from_cancer']
  for column in categorical_columns:
    most_frequent_class = df[column].mode()[0]
    df[column] = df[column].fillna(most_frequent_class)

  # label encoding for cellularity 40 nan values transformed to 0
  mapping = {
      'Low': 1,
      'Moderate': 2,
      'High': 3,
  }
#['pr_status',   'primary_tumor_laterality',   'inferred_menopausal_state',   'her2_status',   'er_status',   'er_status_measured_by_ihc',   '3-gene_classifier_subtype', 'death_from_cancer']

  df['cellularity'] = df['cellularity'].str.strip()
  df["cellularity"] = df["cellularity"].map(mapping)
  df["cellularity"] = df["cellularity"].fillna(3)


  her2_mapping={
  'LOSS' : -1,
  'NEUTRAL' : 0,
  'GAIN' : 1,
  }

  df=df[df.her2_status_measured_by_snp6 != 'UNDEF']
  df['her2_status_measured_by_snp6'] = df['her2_status_measured_by_snp6'].str.strip()
  df['her2_status_measured_by_snp6'] = df['her2_status_measured_by_snp6'].map(her2_mapping)

  label_encoders = {}
  y = df["cancer_type"]
  df = df.drop('cancer_type', axis = 1) 
  label_encoder = LabelEncoder()
  y  = label_encoder.fit_transform(y)
  # def map_values(value):
  #   if value in [1, 2]:
  #       return 1 
  #   else:
  #       return value  

  # y = [map_values(value) for value in y]
  y = pd.Series(y,name='cancer_type')

  df=one_hot_encode_columns(df,cat_columns,isTrain=isTrain)
  

  
  for column in df.columns:
      if df[column].dtype == 'object' and column not in ['pr_status',  'pam50_+_claudin-low_subtype', 'primary_tumor_laterality',   'inferred_menopausal_state',   'her2_status',   'er_status',   'er_status_measured_by_ihc',   '3-gene_classifier_subtype', 'death_from_cancer']:
          
          le = LabelEncoder()

          df[column] = le.fit_transform(df[column].astype(str))

          label_encoders[column] = le
  
  last_seven = df.iloc[:, -7:]
  part_before = df.iloc[:, :2] 
  part_after = df.iloc[:, 2:]
  df = pd.concat([part_before, last_seven, part_after], axis=1)
  df = df.iloc[:, :-7]


  
  return df,y

In [389]:
X, y=data_preprocess(data.copy(),isTrain=True)

SPLITTING

In [397]:
def split_data_by_samples(X, y, desired_samples):
    def get_class_indices(y):
        class_indices = {}
        for i, label in enumerate(np.unique(y)):
            class_indices[label] = np.where(y == label)[0]
        return class_indices
    
    class_indices = get_class_indices(y)
    
    all_indices = np.arange(len(X))
    selected = []
    X_train_selected = pd.DataFrame()
    y_train_selected = []

    X_test_selected = pd.DataFrame()
    y_test_selected = []

    for class_label, num_samples in desired_samples.items():
        indices = class_indices[class_label]
        selected_indices = np.random.choice(indices, size=num_samples, replace=False)
        selected.extend(selected_indices.tolist())
        X_train_selected = pd.concat([X_train_selected, X.loc[selected_indices]])
        y_train_selected.extend(y[selected_indices])

    test_indices = np.setdiff1d(all_indices, sorted(selected))
    X_test_selected = pd.concat([X_test_selected, X.loc[test_indices]])
    y_test_selected.extend(y[test_indices])

    return X_train_selected, pd.Series(y_train_selected), X_test_selected, pd.Series(y_test_selected)



desired_samples = {0: 700, 1: 100,2:12,3:150}
X_train, y_train,X_test, y_test=split_data_by_samples(X,y,desired_samples=desired_samples)
X_validate, X_test, y_validate, y_test =train_test_split(X_test,y_test,shuffle=True,test_size=0.5,random_state=42)

OVERSAMPLING

In [399]:
import imblearn
ROS=imblearn.over_sampling.RandomOverSampler(random_state=0)
X_train, y_train=ROS.fit_resample(X_train,y_train)

TRAINING

In [401]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

xgb_classifier = XGBClassifier(num_class=4,n_estimators=100, random_state=42)

# Train the classifier
xgb_classifier.fit(X_train, y_train)

# Predict on validation set
y_pred = xgb_classifier.predict(X_validate)

# Calculate F1 score
f1 = f1_score(y_validate, y_pred)

print("acc Score on Validation Set:", f1)



acc Score on Validation Set: 0.8411764705882353


LEARNING CURVE PLOT

In [ ]:
from xgboost.callback import TrainingCallback
class EvaluationHistoryCallback(TrainingCallback):
    def __init__(self):
        super().__init__()
        self.evaluation_results = {}

    def after_iteration(self, model, epoch, evals_log):
        if not evals_log:
            return False
        for dataset_name, metric_dict in evals_log.items():
            for metric_name, log in metric_dict.items():
                full_metric_name = f"{dataset_name}-{metric_name}"
                if full_metric_name not in self.evaluation_results:
                    self.evaluation_results[full_metric_name] = []
                self.evaluation_results[full_metric_name].append(log[-1])
        return False  # Return False to continue training

# Initialize callback
eval_history_multi = EvaluationHistoryCallback()
params_multi = {
    'objective': 'multi:softmax',
    'num_class': 4,
    'eval_metric': 'mlogloss',  
    'device': 'cuda',
    'learning_rate': 0.1,
    'max_depth': 15,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'lambda': 1.5,
    'alpha': 0.01,
    'num_parallel_tree': 5
}

dtrain=xgb.DMatrix(X_train,label=y_train)
dval=xgb.DMatrix(X_validate,label=y_validate)
eval_set_multi = [(dtrain, 'train'), (dval, 'eval')]
multi_class_model = xgb.train(params_multi, dtrain, num_boost_round=100, evals=eval_set_multi, early_stopping_rounds=10, verbose_eval=True, callbacks=[eval_history_multi])

# Access the stored evaluation results
results_multi = eval_history_multi.evaluation_results
train_mlogloss = results_multi.get('train-mlogloss', [])
eval_mlogloss = results_multi.get('eval-mlogloss', [])
epochs_multi = len(train_mlogloss)
x_axis_multi = range(0, epochs_multi)

# Plot learning curves
fig, ax = plt.subplots()
ax.plot(x_axis_multi, train_mlogloss, label='Train MLogLoss')
ax.plot(x_axis_multi, eval_mlogloss, label='Validation MLogLoss')
ax.legend()
plt.ylabel('MLogLoss')
plt.title('Learning Curve for Multi-Class Model')
plt.show()

**Confusion Matrix**

In [ ]:
from sklearn.metrics import f1_score
import seaborn as sns
#predicted_classes = np.argmax(predictions, axis=1)

# Compute F1 score
f1 = f1_score(y_validate, y_pred, average='macro')

# Visualize confusion matrix
cm = confusion_matrix(y_validate, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_validate), yticklabels=np.unique(y_validate))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

print(f'F1 Score: {f1:.4f}')

**HyperParameter Tuning**

In [16]:
import cupy as cp
hyper_params={
    'objective':['multi:softmax'],
    'eval_metric':['logloss','mse','mae'],
	'eta':[0.01,0.1,0.3,0.9],
	'gamma':[0.05,0.1,0.2],
	'max_depth':[3,6,12],
	'n_estimators':[100,200,300],
    'num_parallel_tree':[3,10,30],
	'subsample':[0.3,0.5,0.8],
	'sampling_method':['uniform','gradient_based'],
	'lambda':[0.1, 0.5 , 1, 5, 7,10],
    'alpha':[0.01,0.05,0.1,0.3,1],
	'tree_method':['hist','approx'],
	'grow_policy':['depthwise','lossguide'],
}
allParams=sorted(hyper_params)

combinations=list(itr.product(*(hyper_params[param] for param in allParams)))


def grid_search_xgb(parameters, X_train, y_train, cv=2):
  """
  Performs grid search with KFold cross-validation for XGBoost with DMatrix.

  Args:
      params: Dictionary of hyperparameter grids to search over.
      x_train: DMatrix containing training data features.
      y_train: Training data target labels.
      cv: Number of folds for KFold cross-validation (default=5).
      scoring: Evaluation metric for scoring models (default='neg_mean_squared_error').
      eval_metric: Evaluation metric reported during training (default='rmse').

  Returns:
      best_params: Dictionary containing the best hyperparameters found.
      best_model: The XGBoost model with the best hyperparameters.
      cv_results: Dictionary containing cross-validation results for each parameter combination.
  """

  best_params = None
  f1_scores=[float('-inf'),float('-inf'),float('-inf')]

  cv_results = {}


  kfold = KFold(n_splits=cv, shuffle=True)

  for combination in parameters:
    for train_idx, val_idx in kfold.split(X_train, y_train):
      x_train_fold = X_train.iloc[train_idx]
      x_val_fold = X_train.iloc[val_idx]
      y_train_fold = y_train[train_idx]
      y_val_fold = y_train[val_idx]


      train_dmatrix = xgb.DMatrix(x_train_fold, label=y_train_fold)
      val_dmatrix = xgb.DMatrix(x_val_fold, label=y_val_fold)

      params = {
          'num_class': 4,
          'alpha': combination[0],
          'device': 'cuda',
          'eta': combination[1],
          'eval_metric': 'logloss',
          'gamma': combination[3],
          'grow_policy': combination[4],
          'lambda': combination[5],
          'max_depth': combination[6],
          'num_parallel_tree': combination[8],
          'objective': 'multi:softmax',
          'sampling_method': combination[10],
          'subsample': combination[11],
          'tree_method': combination[12],
      }


      model = xgb.train(params, train_dmatrix,
                  num_boost_round=200)

      #eval_results = model.eval(val_dmatrix, name='validation')
      #validation_score = eval_results['validation']['logloss']

      predictions=model.predict(val_dmatrix)
      report=classification_report(y_val_fold,predictions,output_dict=True)
      print(f'{report} for parameters {params}')
      flags=[False,False,False]
      for key, value in report.items():
        if key==1 and f1_scores[key]<value:
          f1_scores[key]=value
          flags[key-1]=True
        elif key==2 and f1_scores[key]<value:
          f1_scores[key]=value
          flags[key-1]=True
        elif key==3 and f1_scores[key]<value:
          f1_scores[key]=value
          flags[key-1]=value

      if flags[0]==True and flags[1]==True and flags[2]==True:
        best_scores = f1_scores
        best_params = combination
        best_model = model

      if combination not in cv_results:
        cv_results[combination] = []
      cv_results[combination].append(f1_scores)

  return best_params, best_model, cv_results


best_params, best_model, cv_results = grid_search_xgb(combinations, X_train, y_train)

# Access results
print(f"Best Hyperparameters: {best_params}")
print(f"Best Model Score: {cv_results}")

c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

{'0': {'precision': 0.7628083491461101, 'recall': 0.9975186104218362, 'f1-score': 0.864516129032258, 'support': 403.0}, '1': {'precision': 0.8333333333333334, 'recall': 0.10416666666666667, 'f1-score': 0.18518518518518517, 'support': 48.0}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 73.0}, 'accuracy': 0.7636022514071295, 'macro avg': {'precision': 0.39903542061986086, 'recall': 0.2754213192721257, 'f1-score': 0.2624253285543608, 'support': 533.0}, 'weighted avg': {'precision': 0.6518044365964022, 'recall': 0.7636022514071295, 'f1-score': 0.6703356264331874, 'support': 533.0}} for parameters {'num_class': 4, 'alpha': 0.01, 'device': 'cuda', 'eta': 0.01, 'eval_metric': 'logloss', 'gamma': 0.05, 'grow_policy': 'depthwise', 'lambda': 0.1, 'max_depth': 3, 'num_parallel_tree': 3, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.3, 'tree_method': 'hist'}


c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

{'0': {'precision': 0.747148288973384, 'recall': 0.9899244332493703, 'f1-score': 0.8515709642470206, 'support': 397.0}, '1': {'precision': 0.3333333333333333, 'recall': 0.038461538461538464, 'f1-score': 0.06896551724137931, 'support': 52.0}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6.0}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 77.0}, 'accuracy': 0.7424812030075187, 'macro avg': {'precision': 0.2701204055766793, 'recall': 0.2570964929277272, 'f1-score': 0.2301341203721, 'support': 532.0}, 'weighted avg': {'precision': 0.5901338422100879, 'recall': 0.7424812030075187, 'f1-score': 0.6422178189898852, 'support': 532.0}} for parameters {'num_class': 4, 'alpha': 0.01, 'device': 'cuda', 'eta': 0.01, 'eval_metric': 'logloss', 'gamma': 0.05, 'grow_policy': 'depthwise', 'lambda': 0.1, 'max_depth': 3, 'num_parallel_tree': 3, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.3, 'tree_method': 'hist'}


c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

...

{'0': {'precision': 0.7833655705996132, 'recall': 0.9878048780487805, 'f1-score': 0.8737864077669902, 'support': 410.0}, '1': {'precision': 0.375, 'recall': 0.14285714285714285, 'f1-score': 0.20689655172413793, 'support': 42.0}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6.0}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 75.0}, 'accuracy': 0.7711069418386491, 'macro avg': {'precision': 0.2895913926499033, 'recall': 0.2826655052264808, 'f1-score': 0.27017073987278206, 'support': 533.0}, 'weighted avg': {'precision': 0.6321386190353497, 'recall': 0.7711069418386491, 'f1-score': 0.6884466835963974, 'support': 533.0}} for parameters {'num_class': 4, 'alpha': 0.01, 'device': 'cuda', 'eta': 0.01, 'eval_metric': 'logloss', 'gamma': 0.05, 'grow_policy': 'depthwise', 'lambda': 0.1, 'max_depth': 3, 'num_parallel_tree': 30, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.8, 'tree_method': 'hist'}


c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

{'0': {'precision': 0.7414448669201521, 'recall': 1.0, 'f1-score': 0.851528384279476, 'support': 390.0}, '1': {'precision': 0.6666666666666666, 'recall': 0.06896551724137931, 'f1-score': 0.125, 'support': 58.0}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 75.0}, 'accuracy': 0.7406015037593985, 'macro avg': {'precision': 0.35202788339670466, 'recall': 0.2672413793103448, 'f1-score': 0.244132096069869, 'support': 532.0}, 'weighted avg': {'precision': 0.6162221142209134, 'recall': 0.7406015037593985, 'f1-score': 0.6378685523853301, 'support': 532.0}} for parameters {'num_class': 4, 'alpha': 0.01, 'device': 'cuda', 'eta': 0.01, 'eval_metric': 'logloss', 'gamma': 0.05, 'grow_policy': 'depthwise', 'lambda': 0.1, 'max_depth': 3, 'num_parallel_tree': 30, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.8, 'tree_method': 'hist'}


c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Moustapha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

{'0': {'precision': 0.7490494296577946, 'recall': 0.9974683544303797, 'f1-score': 0.8555917480998915, 'support': 395.0}, '1': {'precision': 0.7142857142857143, 'recall': 0.09803921568627451, 'f1-score': 0.1724137931034483, 'support': 51.0}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9.0}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 78.0}, 'accuracy': 0.7485928705440901, 'macro avg': {'precision': 0.36583378598587724, 'recall': 0.2738768925291636, 'f1-score': 0.25700138530083494, 'support': 533.0}, 'weighted avg': {'precision': 0.6234579664979368, 'recall': 0.7485928705440901, 'f1-score': 0.6505663113465909, 'support': 533.0}} for parameters {'num_class': 4, 'alpha': 0.01, 'device': 'cuda', 'eta': 0.01, 'eval_metric': 'logloss', 'gamma': 0.05, 'grow_policy': 'depthwise', 'lambda': 0.1, 'max_depth': 3, 'num_parallel_tree': 30, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.8, 'tree_method': 'approx'}
